In [2]:
import pandas as pd

In [5]:


# Replace 'your_file.csv' with the actual path to your CSV file
file_path = 'Rates_DF_Example.xls'
df = pd.read_excel(file_path)

# Display the first few rows of the dataframe
print("First few rows of the dataframe:")
print(df.head())

# Display general information about the dataframe
print("\nGeneral information about the dataframe:")
print(df.info())

# Check for missing values
print("\nMissing values:")
print(df.isnull().sum())


First few rows of the dataframe:
   service_code service_code_description hospital_ID     cost
0         29870             Knee surgery  1003820630  1432.34
1         29870             Knee surgery  1003820630  1016.51
2         29850              Hip Surgery  1003820630  1519.18
3         29850              Hip Surgery  1003820630  1519.18
4         22818            Covid Vaccine  1003820630  5402.73

General information about the dataframe:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10754 entries, 0 to 10753
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   service_code              10754 non-null  int64  
 1   service_code_description  10754 non-null  object 
 2   hospital_ID               10754 non-null  object 
 3   cost                      10754 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 336.2+ KB
None

Missing values:
service_code                0

In [7]:

# Remove rows with missing values
df_cleaned = df.dropna()

# Display the shape of the cleaned dataframe
print("\nShape of the cleaned dataframe:", df_cleaned.shape)

# Display descriptive statistics for numeric columns
print("\nDescriptive statistics for numeric columns:")
print(df_cleaned.describe())



Shape of the cleaned dataframe: (10754, 4)

Descriptive statistics for numeric columns:
       service_code          cost
count  10754.000000  10754.000000
mean   27513.814767   1769.371624
std     3319.449052   1632.969857
min    22818.000000    351.220000
25%    22818.000000    701.460000
50%    29850.000000   1002.290000
75%    29870.000000   2501.200000
max    29870.000000  20843.360000


In [9]:

# Calculate the mean, median, and standard deviation of the 'Cost' column
mean_cost = df_cleaned['cost'].mean()
median_cost = df_cleaned['cost'].median()
std_dev_cost = df_cleaned['cost'].std()

print("\nMean Cost:", mean_cost)
print("Median Cost:", median_cost)
print("Standard Deviation of Cost:", std_dev_cost)



Mean Cost: 1769.3716235819227
Median Cost: 1002.29
Standard Deviation of Cost: 1632.9698565730098


In [11]:

# Group data by 'Hospital ID' and calculate statistics
grouped = df_cleaned.groupby('hospital_ID')
hospital_stats = grouped['cost'].agg(['mean', 'median', 'std', 'count'])

print("\nStatistics grouped by hospital_ID:")
print(hospital_stats)



Statistics grouped by hospital_ID:
                    mean   median          std  count
hospital_ID                                          
1003820630   2715.445000  1519.18  2089.872715      6
1013970078   2715.445000  1519.18  2089.872715      6
1043378979   1217.766667   698.56   917.625824      6
1063475580   2715.445000  1519.18  2089.872715      6
1114358348   2384.770000  2944.26  1106.871974      6
...                  ...      ...          ...    ...
90-0811673   1277.945000   612.71  1094.183392      6
90-0907893   1267.520000   701.46  1012.857880      3
94-3283324   1537.285000   875.14  1160.813960      6
95-4565259   2715.445000  1519.18  2089.872715      6
95-4662001   1189.283333   664.62   945.262202      3

[1463 rows x 4 columns]


In [23]:


def calculate_rankings(df):
    # Group data by 'service_code_description' and calculate rankings within each group
    df['Rank'] = df.groupby('service_code_description')['cost'].rank(ascending=True)
    
    # Create a list to store rows for the result dataframe
    result_rows = []
    
    # Iterate over unique service code descriptions and find costliest and cheapest hospital IDs
    for service_code_description in df['service_code_description'].unique():
        service_df = df[df['service_code_description'] == service_code_description]
        
        costliest_hospital = service_df.loc[service_df['Rank'].idxmax()]
        cheapest_hospital = service_df.loc[service_df['Rank'].idxmin()]
        
        result_rows.append({
            'Service Code Description': service_code_description,
            'Service Code': costliest_hospital['service_code'],
            'Costliest Hospital ID': costliest_hospital['hospital_ID'],
            'Cheapest Hospital ID': cheapest_hospital['hospital_ID']
        })
    
    # Create the result dataframe using pd.DataFrame
    result_df = pd.DataFrame(result_rows)
    
    return result_df

# Replace 'Rates_DF_Example.xls' with the actual path to your Excel file
file_path = 'Rates_DF_Example.xls'
df = pd.read_excel(file_path)

# Clean the data (drop rows with missing values)
df_cleaned = df.dropna(subset=['cost', 'service_code', 'service_code_description', 'hospital_ID'])

# Calculate rankings and costliest/cheapest hospital IDs
result = calculate_rankings(df_cleaned)

print(result)


  Service Code Description  Service Code Costliest Hospital ID  \
0             Knee surgery         29870            20-8624691   
1              Hip Surgery         29850            20-8624691   
2            Covid Vaccine         22818            20-8624691   

  Cheapest Hospital ID  
0           20-8075502  
1           20-8075502  
2           27-1145142  


In [44]:

import sqlite3

# Replace 'Rates_DF_Example.xls' with the actual path to your Excel file
xls_file_path = 'Rates_DF_Example.xls'

# Read the XLS file into a Pandas DataFrame
df = pd.read_excel(xls_file_path)

# Replace 'your_database.db' with the desired name for your SQLite database
db_file = 'Rates_DF.db'

# Create a connection to the SQLite database
conn = sqlite3.connect(db_file)

# Write the DataFrame to the SQLite database
df.to_sql('DF_Table', conn, if_exists='replace', index=False)


# Replace 'your_database.db' with the actual name of your SQLite database file
db_file = 'Rates_DF.db'


# Create a connection to the SQLite database
conn = sqlite3.connect(db_file)

# SQL query to calculate mean of the cost across each service code
mean_query = '''
SELECT
    service_code,
    AVG(cost) AS mean_cost
FROM
    "DF_Table"
GROUP BY
    service_code;
'''

# Execute the mean query and fetch the results
mean_results = conn.execute(mean_query).fetchall()

# SQL query to calculate standard deviation of the cost across each service code
std_dev_query = '''
SELECT
    "DF_Table".service_code,
    SQRT(AVG(("DF_Table".cost - mean_subquery.mean_cost) * ("DF_Table".cost - mean_subquery.mean_cost))) AS std_dev_cost
FROM
    "DF_Table"
JOIN (
    SELECT
        service_code,
        AVG(cost) AS mean_cost
    FROM
        "DF_Table"
    GROUP BY
        service_code
) AS mean_subquery
ON "DF_Table".service_code = mean_subquery.service_code
GROUP BY
    "DF_Table".service_code;
'''

# Execute the standard deviation query and fetch the results
std_dev_results = conn.execute(std_dev_query).fetchall()

# Combine the results into a dictionary for easy printing
results_dict = {}
for row in mean_results:
    results_dict[row[0]] = {'mean_cost': row[1]}

for row in std_dev_results:
    results_dict[row[0]]['std_dev_cost'] = row[1]

# Print the results
for service_code, stats in results_dict.items():
    print("Service Code:", service_code)
    print("Mean Cost:", stats['mean_cost'])
    print("Standard Deviation:", stats['std_dev_cost'])
    print("-" * 20)

# Close the database connection
conn.close()



Service Code: 22818
Mean Cost: 3510.2164387385487
Standard Deviation: 1717.3378553213065
--------------------
Service Code: 29850
Mean Cost: 971.4395021097149
Standard Deviation: 493.6001521488956
--------------------
Service Code: 29870
Mean Cost: 828.8853733406972
Standard Deviation: 504.4764862143422
--------------------
